# LangChain: Q&A over Documents

사람들이 LLM을 사용하여 구축하는 가장 일반적이고 복잡한 애플리케이션 중 하나는 **문서에 관한 질문에 답할 수 있는 시스템**입니다. PDF 파일이나 웹 페이지 또는 일부 회사의 인트라넷 내부 문서 컬렉션에서 추출된 텍스트가 주어지면, LLM을 사용하여 해당 문서의 내용에 대한 질문에 답하여 사용자가 더 깊이 이해하고 얻을 수 있도록 도울 수 있거나, 필요한 정보에 접근할 수 있게됩니다. 이는 언어 모델을 원래 훈련되지 않은 데이터와 결합하기 시작하기 때문에 정말 강력합니다. 따라서 사용 사례에 훨씬 더 유연하게 적용될 수 있습니다. 또한 우리는 언어 모델, 프롬프트 및 출력 파서를 넘어 임베딩 모델 및 벡터 저장소와 같은 LangChain의 주요 구성 요소를 더 많이 도입하기 시작하기 때문에 정말 흥미로울 것입니다.

관심 있는 항목에 대한 제품 카탈로그를 쿼리할 수 있는 도구를 예로 들 수 있습니다.

In [163]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [165]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [167]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')

In [169]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings

In [171]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings()
).from_loaders([loader])

In [173]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [175]:
from langchain_openai import OpenAI

In [177]:
llm_replacement_model = OpenAI(temperature=0, model='gpt-3.5-turbo-instruct')
response = index.query(query, llm = llm_replacement_model)

In [179]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rating, front and back cape venting, two front bellows pockets | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rating, wicks moisture, abrasion resistant | SPF 50+, blocks 98% of harmful UV rays |

### LLM's on Documents

우리는 언어 모델을 사용하고 이를 많은 문서와 결합하여 활용하고 싶습니다.
하지만 중요한 문제가 있습니다.
언어 모델은 한 번에 수천 단어만 검사할 수 있습니다.
그렇다면 매우 큰 문서가 있는 경우 언어 모델이 거기에 있는 모든 것에 대한 질문에 답하도록 하려면 어떻게 해야 할까요?
여기서는 **임베딩(Embedding)** 과 **벡터 저장소(Vector Store)** 가 작동합니다. 먼저 임베딩(Embedding)에 대해 알아보겠습니다.

<img src="./LLM.png" width="220">

### Embeddings

임베딩은 텍스트 조각에 대한 숫자 표현을 만듭니다.
**이 숫자 표현은 지나간 텍스트 조각의 의미를 포착**합니다.
유사한 내용을 가진 텍스트 조각은 유사한 벡터를 갖습니다.
이를 통해 벡터 공간의 텍스트 조각을 비교할 수 있습니다.

<img src="./Embedding-1.png" width="400">

아래 예에서는 세 개의 문장이 있음을 알 수 있습니다.
처음 두 개는 애완동물에 관한 것이고, 세 번째는 자동차에 관한 것입니다.
숫자 공간의 표현을 보면 애완동물에 관한 문장에 해당하는 텍스트 조각의 두 벡터를 비교할 때 매우 유사하다는 것을 알 수 있습니다.
반면에 자동차에 대해 이야기하는 것과 비교하면 전혀 유사하지 않습니다.
이를 통해 어떤 텍스트 조각이 서로 유사한지 쉽게 파악할 수 있으며, 질문에 답하기 위해 언어 모델에 전달할 때 어떤 텍스트 조각을 포함할지 생각할 때 매우 유용합니다.

<img src="./Embedding-2.png" width="450">

### Vector Database

우리가 다룰 다음 구성 요소는 벡터 데이터베이스입니다.
**벡터 데이터베이스**는 이전 단계에서 생성한 이러한 **벡터 표현을 저장**하는 방법입니다.
이 벡터 데이터베이스를 만드는 방법은 들어오는 문서에서 나오는 텍스트 덩어리로 채우는 것입니다.
큰 문서가 수신되면 먼저 이를 더 작은 덩어리로 나눕니다.
이는 원본 문서보다 작은 텍스트 조각을 만드는 데 도움이 되며, 전체 문서를 언어 모델에 전달할 수 없기 때문에 유용합니다. 그래서 우리는 이러한 작은 덩어리를 만들고 이를 벡터 데이터베이스에 저장하기 위해서 각 청크에 대한 임베딩을 생성합니다.

<img src="./VectorDB.PNG" width="400">

인덱스를 생성하면 어떤 일이 발생합니까? 이제 이 인덱스가 있으므로 런타임 중에 이를 사용하여 들어오는 쿼리와 가장 관련성이 높은 텍스트 조각을 찾을 수 있습니다.
쿼리가 들어오면 먼저 해당 쿼리에 대한 임베딩을 만듭니다.
그런 다음 이를 벡터 데이터베이스의 모든 벡터와 비교하고 가장 유사한 n개를 선택합니다.
그런 다음 이러한 내용이 반환되고 프롬프트의 내용을 언어 모델에 전달하여 최종 답변을 얻을 수 있습니다. 

<img src="./VectorDB2.png" width="400">

## Step By Step

In [190]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [192]:
docs = loader.load()

In [194]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [196]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [198]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [200]:
print(len(embed))

1536


In [202]:
print(embed[:5])

[-0.021878015249967575, 0.0067466627806425095, -0.018212852999567986, -0.03916202485561371, -0.014020507223904133]


방금 로드한 모든 텍스트 조각에 대한 임베딩을 만들고 이를 벡터 저장소에 저장하려고 합니다. 벡터 저장소의 "from_documents" 메소드를 사용하면 이를 수행할 수 있습니다. 이 메서드는 문서 목록과 임베딩 개체를 가져온 다음 전체 벡터 저장소를 만듭니다. 이제 이 벡터 저장소를 사용하여 들어오는 쿼리와 유사한 텍스트 조각을 찾을 수 있습니다.

In [205]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [206]:
query = "Please suggest a shirt with sunblocking"

In [207]:
docs = db.similarity_search(query)

In [208]:
len(docs)

4

In [209]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

그렇다면 이를 어떻게 사용하여 문서에 대한 질문에 답할 수 있을까요? 먼저 이 벡터 저장소에서 검색기(retriever)를 만들어야 합니다. 검색기(retriever)는 쿼리를 받아 문서를 반환하는 모든 메서드로 뒷받침될 수 있는 일반 인터페이스입니다. 벡터 스토어와 임베딩은 그렇게 하기 위한 방법 중 하나입니다. 물론 덜 발전된 방법도 있고 좀 더 발전된 방법도 있습니다. 

In [216]:
retriever = db.as_retriever()

다음으로 텍스트 생성을 수행하고 자연어 응답을 반환하려고 하므로 언어 모델을 가져오고 ChatOpenAl을 사용하겠습니다.

In [219]:
from langchain_openai import ChatOpenAI

In [221]:
llm = ChatOpenAI(temperature = 0.0)

 이 작업을 수동으로 수행한다면 문서를 하나의 텍스트로 결합할 것입니다. 따라서 우리는 문서의 모든 페이지 콘텐츠를 변수로 결합한 다음, 이 변수 와 질문에 대한 변형을 전달하는 다음과 같은 작업을 수행합니다. 

In [224]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [274]:
response = llm.invoke("{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

In [276]:
display(Markdown(response.content))

| Shirt Name          | Sun Protection Level | Summary                                                                                   |
|---------------------|----------------------|-------------------------------------------------------------------------------------------|
| Coolibar Men's UPF 50+ T-Shirt | UPF 50+ | This shirt offers maximum sun protection with a UPF rating of 50+. It is made with breathable fabric and is perfect for outdoor activities. |
| Columbia Women's Sun Goddess II Long Sleeve Shirt | UPF 30 | This shirt has a UPF rating of 30, providing good sun protection. It features long sleeves and a lightweight design, making it ideal for hiking or fishing. |
| Patagonia Men's Sunshade Technical Hoody | UPF 50+ | With a UPF rating of 50+, this hoody offers excellent sun protection. It has a hood for added coverage and is made with moisture-wicking fabric for comfort during outdoor adventures. |

### Using LangChain

이러한 모든 단계는 LangChain 체인으로 캡슐화될 수 있습니다.
따라서 여기에서 RetrivalQA 체인을 만들 수 있습니다. 이는 검색을 수행한 다음, 검색된 문서에 대한 질문 답변을 수행합니다. 이러한 체인을 만들기 위해 몇 가지 다른 사항을 전달합니다. 먼저 LLM 을 전달하겠습니다.
이는 마지막에 텍스트 생성을 수행하는 데 사용됩니다.
다음으로 chain_type을 전달하겠습니다. 우리는 "stuff"를 사용할 것입니다. 이는 모든 문서를 컨텍스트에 넣고 언어 모델을 한 번 호출하기 때문에 가장 간단한 방법입니다.
질문 답변을 수행하는 데 사용할 수 있는 몇 가지 다른 방법이 있으며 마지막에 다루겠지만 자세히 살펴보진 않겠습니다. 셋째, retriever를 전달할 것입니다.위에서 만든 retriever는 문서를 가져오기 위한 인터페이스일 뿐입니다.
이는 문서를 가져와 언어 모델에 전달하는 데 사용됩니다.

In [279]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [281]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [283]:
response = qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [261]:
response['result']

'| Shirt Name                           | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [263]:
display(Markdown(response['result']))

| Shirt Name                           | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

이것이 세부적으로 수행하는 방법입니다. 하지만 위에 있는 한 줄만 있으면 여전히 매우 쉽게 수행할 수 있다는 점을 기억하십시오. 따라서 이 두 가지는 동일한 결과입니다. 그리고 그것은 LangChain의 흥미로운 부분 중 하나입니다. 한 줄로 할 수도 있고, 개별 항목을 살펴보고 5개의 세부 항목으로 나눌 수도 있습니다. 다섯 가지 더 자세한 내용을 사용하면 정확히 무슨 일이 일어나고 있는지에 대해 더 구체적으로 설정할 수 있지만 한 줄만 사용하면 시작하기 쉽습니다. 따라서 앞으로 어떻게 진행하고 싶은지는 여러분에게 달려 있습니다.

In [288]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))

인덱스를 생성할 때 인덱스를 사용자 정의할 수도 있습니다. 기억하시겠지만, 우리가 직접 만들 때 임베딩을 지정했습니다. 그리고 여기서도 임베딩을 지정할 수 있습니다. 따라서 이는 임베딩 자체가 생성되는 방식에 대한 유연성을 제공합니다. 그리고 여기에서 벡터 저장소를 다른 유형의 벡터 저장소로 교체할 수도 있습니다. 따라서 여기에서 색인을 만들 때에도 사용할 수 있는 손으로 만들 때와 동일한 수준의 사용자 지정이 가능합니다.

In [287]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

<img src="Stuff.PNG" width="350">

따라서 우리가 노트북에 있는 문서를 가져왔을 때 4개의 문서만 돌려받았고 그 크기는 상대적으로 작았습니다. 
하지만 다양한 유형의 청크에 대해 동일한 유형의 질문에 답하고 싶다면 어떻게 해야 할까요?
그러면 우리가 사용할 수 있는 몇 가지 다른 방법이 있습니다. 

첫 번째는 "**Map_reduce**"입니다.
이는 기본적으로 모든 청크를 가져와 질문과 함께 언어 모델에 전달하고 응답을 받은 다음 다른 언어 모델 호출을 사용하여 **모든 개별 응답을 최종 답변으로 요약**합니다.
이는 문서 수에 관계없이 작동할 수 있기 때문에 정말 강력합니다.
또한 **개별 질문을 동시에 수행할 수 있기 때문에 매우 강력**합니다.
하지만 **훨씬 더 많은 대화가 필요**합니다. 그리고 모든 문서를 독립적으로 처리하는데, 이것이 항상 가장 바람직한 것은 아닐 수도 있습니다. 

또 다른 방법인 "**Refine**"은 많은 문서를 반복하는 데에도 사용됩니다.
실제로는 반복적으로 수행됩니다. **이전 문서의 답변을 바탕으로 작성**되었습니다.
따라서 이는 정보를 결합하고 시간이 지남에 따라 답변을 구축하는 데 정말 좋습니다. **일반적으로 답변이 길어집**니다.
그리고 이제 **호출이 독립적이지 않기 때문에 속도도 빠르지 않습니다**.
이전 호출의 결과에 따라 달라집니다.
이는 기본적으로 "Map_reduce"만큼 **시간이 오래 걸리고 많은 호출이 필요**하다는 것을 의미합니다.


"**Map_rerank**"는 매우 흥미롭고 각 문서의 언어 모델을 단일 호출하는 좀 더 실험적인 것입니다. 그리고 점수를 반환하도록 요청합니다.
그리고 **가장 높은 점수를 선택**합니다.
이는 점수가 무엇인지 알기 위해 언어 모델에 의존합니다. 그래서 종종 "문서와 관련이 있고 거기에 있는 지침을 실제로 다듬으면 높은 점수를 받아야 해"라고 말해야 합니다. "Map_reduce"와 마찬가지로 **모든 호출은 독립적**입니다. 따라서 일괄 처리할 수 있으며 상대적으로 빠릅니다. 하지만 다시 말하지만, 언어 모델을 여러 번 호출하고 있습니다. 그래서 조금 더 비쌀 것입니다.
이러한 방법 중 **가장 일반적인 것은 "stuff method**"입니다.
우리는 모든 것을 하나의 문서로 결합하기 위해 노트북에서 사용했습니다.
**두 번째로 가장 일반적인 방법**은 이러한 청크를 가져와서 언어 모델로 보내는 "**Map_reduce**" 메서드입니다.

여기에 있는 이러한 방법, stuff, map_reduce, Refine 및 Map_rerank는 질문 답변 외에도 많은 다른 체인에도 사용될 수 있습니다.
예를 들어, "Map_reduce" 체인의 가장 일반적인 사용 사례는 요약을 위한 것인데, 매우 긴 문서가 있고 그 안에 있는 정보를 반복적으로 요약하려는 경우입니다.


<img src="./Map.PNG" width="400">